In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd
import numpy as np
# import urllib

In [2]:
## Crawl_GroupPosts
def Crawl_GroupPosts(groupurl, until_date='2019-01-01'):
    # init parameters
    rs = requests.Session()
    content_df = [] # post
    feedback_df = [] # reactions
    bac = ''
    break_times = 0
    max_date =  datetime.datetime.now().strftime('%Y-%m-%d')
    headers = {'sec-fetch-site': 'same-origin',
               'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
               'x-fb-lsd': 'GoogleBot'}
    data = {'lsd': 'GoogleBot', 
            '__a': 'GoogleBot'}
    
    # redirect to m.facebook
    groupurl = re.sub('www','m', groupurl)

    # request data and break loop until reach the goal 
    while max_date >= until_date:
        
        # request params
        params = {
            'bac': bac,
            'multi_permalinks': '',
            'refid': '18'
            }
        resp = rs.post(groupurl, headers=headers, params=params, data=data)
        resp = re.sub(r'for \(;;\);', '', resp.text)
        
        try:
            resp = json.loads(resp)
            soup = BeautifulSoup(resp['payload']['actions'][0]['html'], "lxml")
            reactions = re.findall('\(new \(require\("ServerJS"\)\)\(\)\).handle\((.*?)\);', resp['payload']['actions'][2]['code'])[0]
            # Parse content
            for post in soup.select('section > article'):
                try:
                    content_df.append([
                        re.findall('"content_owner_id_new":(.*?),', str(post))[0], # ACTORID
                        post.select('strong > a')[0].text, # NAME
                        re.findall('"page_id":"(.*?)"', str(post))[0], # GROUPID
                        re.findall('"mf_story_key":"(.*?)"', str(post))[0], # POSTID
                        re.findall('"publish_time":(.*?),', str(post))[0], # TIME
                        post.find('div',{'data-ft':'{"tn":"*s"}'}).text # CONTENT
                    ])
                except:
                    pass
            # Parse reaction
            for ele in json.loads(reactions)['require']:
                if 'counts' in str(ele):
                    feedback_df.append([
                        ele[3][1]['ft_ent_identifier'], # POSTID
                        ele[3][1]['comment_count'], # comment_count
                        ele[3][1]['share_count'], # # share_count
                        ele[3][1]['like_count'] # like_count
                    ])
             # Update information
            max_date = max([re.findall('"publish_time":(.*?),', str(time['data-ft']))[0] for time in soup.select('section > article')])
            max_date = datetime.datetime.fromtimestamp(int(max_date)).strftime('%Y-%m-%d')
            print(f'TimeStamp: {max_date}.')
            try:
                bac = re.findall('bac=(.*?)%3D', soup.select('div > a.primary')[0]['href'])[0]
            except:
                bac = re.findall('bac=(.*?)&', soup.select('div > a.primary')[0]['href'])[0]
            break_times = 0 # reset break times to zero
            
        except:
            break_times += 1
            print('break_times:', break_times)
            if break_times > 5:
                return resp
                # return soup.select('div > a.primary')[0]['href']
                # return print('ERROR: Please send the following URL to the author. \n', rs.url)
        # time.sleep(4)    
    # join content and reactions
    content_df = pd.DataFrame(content_df, columns=['ACTORID','NAME', 'GROUPID', 'POSTID','TIME', 'CONTENT'])
    content_df['ACTORID'] = content_df['ACTORID'].apply(lambda x: re.sub('"', '', x))
    content_df['TIME'] = content_df['TIME'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime("%Y-%m-%d %H:%M:%S"))
    
    feedback_df = pd.DataFrame(feedback_df, columns=['POSTID', 'COMMENTCOUNT',  'SHARECOUNT', 'LIKECOUNT'])
    feedback_df['POSTID'] = feedback_df['POSTID'].apply(lambda x: str(x))
    
    df = pd.merge(left=content_df, right=feedback_df, how='left', on='POSTID')
    df['UPDATETIME'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  
    return df

In [3]:
groupurl = 'https://www.facebook.com/groups/pythontw'
groupurl = 'https://www.facebook.com/groups/corollacrossclub'
df = Crawl_GroupPosts(groupurl, until_date='2022-01-01')
df

TimeStamp: 2022-04-20.
TimeStamp: 2022-04-20.
TimeStamp: 2022-04-20.
TimeStamp: 2022-04-19.
TimeStamp: 2022-04-19.
TimeStamp: 2022-04-17.
TimeStamp: 2022-04-17.
TimeStamp: 2022-04-17.
TimeStamp: 2022-04-16.
TimeStamp: 2022-04-15.
TimeStamp: 2022-04-15.
TimeStamp: 2022-04-14.
TimeStamp: 2022-04-13.
TimeStamp: 2022-04-12.
TimeStamp: 2022-04-12.
TimeStamp: 2022-04-11.
TimeStamp: 2022-04-11.
TimeStamp: 2022-04-10.
TimeStamp: 2022-04-09.
TimeStamp: 2022-04-08.
TimeStamp: 2022-04-07.
TimeStamp: 2022-04-07.
TimeStamp: 2022-04-06.
TimeStamp: 2022-04-05.
TimeStamp: 2022-04-05.
TimeStamp: 2022-04-04.
TimeStamp: 2022-04-03.
TimeStamp: 2022-04-02.
TimeStamp: 2022-04-01.
TimeStamp: 2022-04-01.
TimeStamp: 2022-03-30.
TimeStamp: 2022-03-30.
TimeStamp: 2022-03-29.
TimeStamp: 2022-03-29.
TimeStamp: 2022-03-28.
TimeStamp: 2022-03-27.
TimeStamp: 2022-03-27.
TimeStamp: 2022-03-25.
TimeStamp: 2022-03-25.
TimeStamp: 2022-03-23.
TimeStamp: 2022-03-22.
TimeStamp: 2022-03-21.
TimeStamp: 2022-03-20.
TimeStamp: 

,ACTORID,NAME,GROUPID,POSTID,TIME,CONTENT,COMMENTCOUNT,SHARECOUNT,LIKECOUNT,UPDATETIME
0,100004202555178,夢吉陳,262151978368069,682552559661340,2022-04-20 21:41:43,有人買安心版的嗎😅 有沒有比較好的不買豪華、尊爵的理由⋯⋯有人買安心版的嗎😅 有沒有比較...,8,0,10,2022-04-20 22:35:08
1,1087967521,簡安,262151978368069,682518366331426,2022-04-20 20:29:46,請問各位大大，只有假日才會開車出去，要選汽油版，還是油電版？？？哪個會比較好？？？,77,0,41,2022-04-20 22:35:08
2,100000256578431,黃瓊瑩,262151978368069,680641083185821,2022-04-17 20:56:53,"新購車,業務建議隔熱紙貼3M8803C 可是鄰居說太黑了,左右後視鏡看不清楚後面 可否請問大...",69,0,64,2022-04-20 22:35:08
3,100000299751443,Lin Ka,262151978368069,682493169667279,2022-04-20 19:34:31,今日美拍 #暴走鬼滅 #台南北門 #夕陽海岸,6,0,37,2022-04-20 22:35:08
4,100000298801689,馮中,271460359650836,594173051832625,2021-11-26 23:50:31,社團特約商 台南隆田汽車 底盤強化四件組 #優惠中 #優惠中 #優惠中 TAXI 🚖 大哥也...,88,1,82,2022-04-20 22:35:08
...,...,...,...,...,...,...,...,...,...,...
2775,100002829954124,李科廷,262151978368069,611558940094036,2021-12-25 22:51:04,聖誕節交車日～ 祝車友們聖誕快樂。 #CorollaCrossGr,37,0,265,2022-04-20 22:35:08
2776,100000059846629,陳志銘,262151978368069,614785553104708,2021-12-31 11:41:12,煩請有裝過四門升降窗飾板的先進們，分享一下產品及店家~感恩！,4,0,5,2022-04-20 22:35:08
2777,100000122735021,Jing Liang Song,262151978368069,613123529937577,2021-12-28 14:53:22,請教一下 這乙式車險 可以嗎？ 是否可拿掉駕駛傷害險和代步車費用保險？,40,1,41,2022-04-20 22:35:08
2778,100000832633198,鄭莓,262151978368069,613665486550048,2021-12-29 13:23:16,變獨眼土虱啦！ 回原廠買一顆＄230 3秒拆 5秒裝 收工 ～ ps 鹵素燈泡手指不要...,32,1,125,2022-04-20 22:35:08
